In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [3]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [4]:
print(start_date,end_date)

2023-02-14 11:17:43.824924 2023-05-14 11:17:43.826764


In [5]:
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

In [7]:
df_list=[]
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
df_list[0]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-14,152.119995,153.770004,150.860001,153.199997,152.988388,61707600
2023-02-15,153.110001,155.500000,152.880005,155.330002,155.115448,65573800
2023-02-16,153.509995,156.330002,153.350006,153.710007,153.497696,68167900
2023-02-17,152.350006,153.000000,150.850006,152.550003,152.339294,59144100
2023-02-21,150.199997,151.300003,148.410004,148.479996,148.274902,58867200
...,...,...,...,...,...,...
2023-05-08,172.479996,173.850006,172.110001,173.500000,173.260345,55962800
2023-05-09,173.050003,173.539993,171.600006,171.770004,171.532745,45326900
2023-05-10,173.020004,174.029999,171.899994,173.559998,173.320267,53724500


In [9]:
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-02-14  152.119995  153.770004  150.860001  153.199997  152.988388   
       2023-02-15  153.110001  155.500000  152.880005  155.330002  155.115448   
       2023-02-16  153.509995  156.330002  153.350006  153.710007  153.497696   
       2023-02-17  152.350006  153.000000  150.850006  152.550003  152.339294   
       2023-02-21  150.199997  151.300003  148.410004  148.479996  148.274902   

                     Volume  
Ticker Date                  
AAPL   2023-02-14  61707600  
       2023-02-15  65573800  
       2023-02-16  68167900  
       2023-02-17  59144100  
       2023-02-21  58867200  


In [10]:
df.describe


<bound method NDFrame.describe of                          Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-02-14  152.119995  153.770004  150.860001  153.199997  152.988388   
       2023-02-15  153.110001  155.500000  152.880005  155.330002  155.115448   
       2023-02-16  153.509995  156.330002  153.350006  153.710007  153.497696   
       2023-02-17  152.350006  153.000000  150.850006  152.550003  152.339294   
       2023-02-21  150.199997  151.300003  148.410004  148.479996  148.274902   
...                       ...         ...         ...         ...         ...   
GOOG   2023-05-08  105.794998  108.419998  105.790001  108.239998  108.239998   
       2023-05-09  108.779999  110.595001  107.724998  107.940002  107.940002   
       2023-05-10  108.550003  113.510002  108.480003  112.279999  112.279999   
       2023-05-11  115.860001  118.440002  114.930000  116.900002  116.9000

In [11]:
df=df.reset_index()

In [12]:
df.head()

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-02-14,152.119995,153.770004,150.860001,153.199997,152.988388,61707600
1,AAPL,2023-02-15,153.110001,155.500000,152.880005,155.330002,155.115448,65573800
2,AAPL,2023-02-16,153.509995,156.330002,153.350006,153.710007,153.497696,68167900
3,AAPL,2023-02-17,152.350006,153.000000,150.850006,152.550003,152.339294,59144100
4,AAPL,2023-02-21,150.199997,151.300003,148.410004,148.479996,148.274902,58867200


In [13]:
import plotly.express  as px
fig=px.line(df,x='Date',y='Close',color='Ticker',title="Stock market performance for the Last 3 Months")
fig.show()

# let’s look at the faceted area chart 
# to identify similarities or differences in their stock price movements

In [16]:
fig=px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')

fig.show()

In [17]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10      MA20
0          NaN       NaN
1          NaN       NaN
2          NaN       NaN
3          NaN       NaN
4          NaN       NaN
..         ...       ...
57  168.405998  166.6030
58  169.205998  167.1515
59  170.185999  167.8245
60  170.719998  168.2340
61  171.009000  168.6020

[62 rows x 2 columns]
Moving Averages for GOOG
         MA10       MA20
186       NaN        NaN
187       NaN        NaN
188       NaN        NaN
189       NaN        NaN
190       NaN        NaN
..        ...        ...
243  106.5125  106.46325
244  106.8455  106.55425
245  107.6285  106.90725
246  108.4815  107.34275
247  109.4515  107.76575

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  302.295999  294.225998
120  305.453998  295.434499
121  307.147998  296.875499

In [18]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [20]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

In [21]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

In [22]:
apple.head()

,Date,AAPL
0,2023-02-14,153.199997
1,2023-02-15,155.330002
2,2023-02-16,153.710007
3,2023-02-17,152.550003
4,2023-02-21,148.479996


In [23]:
# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()